In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil

db = database('trfc.db')

# Player Season Stats

In [110]:
@dataclass
class PlayerSeasonRecord:
    player_name: str
    starts: int
    sub_apps: int
    total_apps: int
    goals_scored: int
    mins_played: Optional[int]
    mins_per_goal: Optional[float]
    games_per_goal: Optional[float]
    win_pc: float
    yellow_cards: int
    red_cards: int

In [109]:
query = """
    SELECT 
        p.surname || ', ' || p.forename AS player,
        COUNT(DISTINCT CASE WHEN pa.role = 'starter' THEN r.game_date END) AS starts,
        COUNT(DISTINCT CASE WHEN pa.role = 'sub' THEN r.game_date END) AS sub_apps,
        COUNT(DISTINCT r.game_date) AS total_apps,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN 0
            ELSE
            SUM(match_goals.goals_scored) END AS goals,
        SUM(
            COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
        ) as mins_played,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN NULL
            ELSE 
            ROUND(CAST(SUM(
                COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
            ) AS FLOAT) / SUM(match_goals.goals_scored), 1)
            END as mins_per_goal,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN NULL
            ELSE 
            ROUND(CAST(SUM(
                COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
            ) AS FLOAT) / SUM(match_goals.goals_scored) / 90, 1)
            END as games_per_goal,
        ROUND(
            CAST(COUNT(DISTINCT CASE WHEN pa.role = 'starter' AND r.outcome = 'W' THEN r.game_date END) AS FLOAT) * 100 / 
            NULLIF(COUNT(DISTINCT CASE WHEN pa.role = 'starter' THEN r.game_date END), 0), 1
        ) as win_pct_as_starter,
        COUNT(yc.player_id) AS yellow_cards,
        COUNT(rc.player_id) AS red_cards
    FROM results r
    JOIN player_apps pa ON pa.game_date = r.game_date
    JOIN players p ON p.player_id = pa.player_id
    LEFT JOIN subs_on ON subs_on.game_date = r.game_date AND subs_on.player_on_id = pa.player_id
    LEFT JOIN subs_off ON subs_off.game_date = r.game_date AND subs_off.player_off_id = pa.player_id
    LEFT JOIN cards_red rc ON rc.game_date = r.game_date AND rc.player_id = pa.player_id
    LEFT JOIN cards_yellow yc ON yc.game_date = r.game_date AND yc.player_id = pa.player_id
    LEFT JOIN cup_game_details cgd ON cgd.game_date = r.game_date
    LEFT JOIN (
        SELECT 
            game_date,
            player_id,
            COUNT(player_id) AS goals_scored
        FROM goals
        GROUP BY game_date, player_id
    ) AS match_goals ON match_goals.game_date = r.game_date AND match_goals.player_id = p.player_id
    WHERE r.season = '2023/24'
    GROUP BY player
"""

db.execute(query).fetchall()

[('Apter, Rob', 33, 7, 40, 13, 2841.0, 218.5, 2.4, 42.4, 4, 0),
 ('Belehouan, Jean', 6, 2, 8, 0, 574.0, None, None, 33.3, 1, 0),
 ('Davies, Mikey', 0, 1, 1, 0, 45.0, None, None, None, 0, 0),
 ('Davies, Tom', 40, 0, 40, 3, 3600.0, 1200.0, 13.3, 32.5, 7, 1),
 ('Dennis, Kristian', 16, 18, 34, 2, 1499.0, 749.5, 8.3, 25.0, 2, 0),
 ('Hawkes, Josh', 13, 26, 39, 6, 1392.0, 232.0, 2.6, 15.4, 0, 0),
 ('Hemmings, Kane', 0, 3, 3, 0, 32.0, None, None, None, 0, 0),
 ('Hendry, Regan', 48, 1, 49, 5, 4313.0, 862.6, 9.6, 35.4, 9, 0),
 ('Hockenhull, Ben', 3, 0, 3, 0, 270, None, None, 66.7, 0, 0),
 ('Hughes, Rhys', 1, 1, 2, 0, 135.0, None, None, 0.0, 0, 0),
 ('Jennings, Connor', 46, 2, 48, 12, 4198.0, 349.8, 3.9, 37.0, 12, 0),
 ('Jolley, Charlie', 12, 17, 29, 4, 1236.0, 309.0, 3.4, 25.0, 3, 0),
 ('Kané, Ousmane', 0, 1, 1, 0, 1.0, None, None, None, 0, 0),
 ('Leake, Jake', 6, 2, 8, 0, 544.0, None, None, 16.7, 2, 0),
 ('Lewis, Paul', 9, 9, 18, 0, 780.0, None, None, 0.0, 3, 0),
 ('McAlear, Reece', 11, 18, 29,

# Player Records by Competition

In [ ]:
@dataclass
class PlayerSeasonRecord:
    player_name: str
    competition: str
    starts: int
    sub_apps: int
    total_apps: int
    goals_scored: int
    mins_played: Optional[int]
    mins_per_goal: Optional[float]
    games_per_goal: Optional[float]
    win_pc: float
    yellow_cards: int
    red_cards: int

In [106]:
query = """
    SELECT 
        p.surname || ', ' || p.forename AS player,
        r.competition,
        COUNT(DISTINCT CASE WHEN pa.role = 'starter' THEN r.game_date END) AS starts,
        COUNT(DISTINCT CASE WHEN pa.role = 'sub' THEN r.game_date END) AS sub_apps,
        COUNT(DISTINCT r.game_date) AS total_apps,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN 0
            ELSE
            SUM(match_goals.goals_scored) END AS goals,
        SUM(
            COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
        ) as mins_played,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN NULL
            ELSE 
            ROUND(CAST(SUM(
                COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
            ) AS FLOAT) / SUM(match_goals.goals_scored), 1)
            END as mins_per_goal,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN NULL
            ELSE 
            ROUND(CAST(SUM(
                COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - COALESCE(subs_on.sub_min_on, 0)
            ) AS FLOAT) / SUM(match_goals.goals_scored) / 90, 1)
            END as games_per_goal,
        ROUND(
            CAST(COUNT(DISTINCT CASE WHEN pa.role = 'starter' AND r.outcome = 'W' THEN r.game_date END) AS FLOAT) * 100 / 
            NULLIF(COUNT(DISTINCT CASE WHEN pa.role = 'starter' THEN r.game_date END), 0), 1
        ) as win_pct_as_starter,
        COUNT(yc.player_id) AS yellow_cards,
        COUNT(rc.player_id) AS red_cards
    FROM results r
    JOIN player_apps pa ON pa.game_date = r.game_date
    JOIN players p ON p.player_id = pa.player_id
    LEFT JOIN subs_on ON subs_on.game_date = r.game_date AND subs_on.player_on_id = pa.player_id
    LEFT JOIN subs_off ON subs_off.game_date = r.game_date AND subs_off.player_off_id = pa.player_id
    LEFT JOIN cards_red rc ON rc.game_date = r.game_date AND rc.player_id = pa.player_id
    LEFT JOIN cards_yellow yc ON yc.game_date = r.game_date AND yc.player_id = pa.player_id
    LEFT JOIN cup_game_details cgd ON cgd.game_date = r.game_date
    LEFT JOIN (
        SELECT 
            game_date,
            player_id,
            COUNT(player_id) AS goals_scored
        FROM goals
        GROUP BY game_date, player_id
    ) AS match_goals ON match_goals.game_date = r.game_date AND match_goals.player_id = p.player_id
    WHERE r.season = '2024/25'
    GROUP BY player, r.competition
"""

db.execute(query).fetchall()

[('Bradshaw, Zak', 'Carabao Cup', 1, 1, 2, 0, 118.0, None, None, 100.0, 0, 0),
 ('Bradshaw, Zak', 'EFL Trophy', 4, 0, 4, 0, 279.0, None, None, 50.0, 1, 1),
 ('Bradshaw, Zak', 'League Two', 5, 7, 12, 0, 529.0, None, None, 20.0, 3, 0),
 ('Davies, Tom', 'Carabao Cup', 1, 0, 1, 0, 90, None, None, 0.0, 1, 0),
 ('Davies, Tom', 'EFL Trophy', 2, 0, 2, 0, 135.0, None, None, 50.0, 0, 0),
 ('Davies, Tom', 'FA Cup', 1, 0, 1, 0, 90, None, None, 0.0, 1, 0),
 ('Davies, Tom', 'League Two', 17, 0, 17, 0, 1494.0, None, None, 29.4, 5, 1),
 ('Davison, Josh', 'Carabao Cup', 1, 1, 2, 0, 91.0, None, None, 0.0, 0, 0),
 ('Davison, Josh', 'EFL Trophy', 3, 1, 4, 1, 239.0, 239.0, 2.7, 33.3, 1, 0),
 ('Davison, Josh', 'League Two', 6, 7, 13, 0, 475.0, None, None, 50.0, 1, 0),
 ('Dennis, Kristian',
  'Carabao Cup',
  1,
  1,
  2,
  0,
  89.0,
  None,
  None,
  100.0,
  0,
  0),
 ('Dennis, Kristian', 'EFL Trophy', 2, 2, 4, 2, 221.0, 110.5, 1.2, 50.0, 0, 0),
 ('Dennis, Kristian', 'FA Cup', 0, 1, 1, 0, 0.0, None, None,

In [112]:
query = """
    SELECT 
        p.surname || ', ' || p.forename AS player,
        r.competition,
        CASE WHEN COUNT(match_goals.player_id) = 0 THEN 0
            ELSE
            SUM(match_goals.goals_scored) END AS goals
    FROM results r
    JOIN player_apps pa ON pa.game_date = r.game_date
    JOIN players p ON p.player_id = pa.player_id
    LEFT JOIN (
        SELECT 
            game_date,
            player_id,
            COUNT(player_id) AS goals_scored
        FROM goals
        GROUP BY game_date, player_id
    ) AS match_goals ON match_goals.game_date = r.game_date AND match_goals.player_id = p.player_id
    WHERE r.season = '2023/24'
    GROUP BY player, competition
"""

db.execute(query).fetchall()

[('Apter, Rob', 'League Two', 12),
 ('Apter, Rob', "Papa John's Trophy", 0),
 ('Apter, Rob', 'The Emirates FA Cup', 1),
 ('Belehouan, Jean', 'League Two', 0),
 ('Belehouan, Jean', "Papa John's Trophy", 0),
 ('Davies, Mikey', "Papa John's Trophy", 0),
 ('Davies, Tom', 'Carabao Cup', 0),
 ('Davies, Tom', 'League Two', 3),
 ('Davies, Tom', 'The Emirates FA Cup', 0),
 ('Dennis, Kristian', 'Carabao Cup', 0),
 ('Dennis, Kristian', 'League Two', 2),
 ('Dennis, Kristian', "Papa John's Trophy", 0),
 ('Dennis, Kristian', 'The Emirates FA Cup', 0),
 ('Hawkes, Josh', 'Carabao Cup', 0),
 ('Hawkes, Josh', 'League Two', 6),
 ('Hawkes, Josh', "Papa John's Trophy", 0),
 ('Hemmings, Kane', 'Carabao Cup', 0),
 ('Hemmings, Kane', 'League Two', 0),
 ('Hendry, Regan', 'Carabao Cup', 0),
 ('Hendry, Regan', 'League Two', 5),
 ('Hendry, Regan', "Papa John's Trophy", 0),
 ('Hendry, Regan', 'The Emirates FA Cup', 0),
 ('Hockenhull, Ben', 'League Two', 0),
 ('Hughes, Rhys', "Papa John's Trophy", 0),
 ('Jennings, 

In [113]:
query = """
    SELECT 
        m.tbl_name AS table_name,
        p.name AS column_name,
        p.type AS data_type,
        CASE WHEN p.pk > 0 THEN 1 ELSE 0 END AS is_primary_key,
        p."notnull" AS is_required,
        p.dflt_value AS default_value
    FROM sqlite_master m
    JOIN pragma_table_info(m.tbl_name) p
    WHERE m.type = 'table'
    ORDER BY m.tbl_name, p.cid;
"""

db.execute(query).fetchall()

[('cards_red', 'game_date', 'TEXT', 1, 0, None),
 ('cards_red', 'player_id', 'TEXT', 1, 0, None),
 ('cards_red', 'min_so', 'FLOAT', 0, 0, None),
 ('cards_yellow', 'game_date', 'TEXT', 1, 0, None),
 ('cards_yellow', 'player_id', 'TEXT', 1, 0, None),
 ('cards_yellow', 'min_yc', 'TEXT', 0, 0, None),
 ('cup_game_details', 'game_date', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_stage', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_round', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_section', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_leg_no', 'TEXT', 1, 0, None),
 ('cup_game_details', 'decider', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_score', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_gf', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_ga', 'TEXT', 1, 0, None),
 ('cup_game_details', 'gg_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'away_go

In [75]:
query = """
    SELECT 
        m.tbl_name || '.' || fk.`from` || ' -> ' || 
        fk.`table` || '.' || fk.`to` as relationship
    FROM sqlite_master m, 
        pragma_foreign_key_list(m.tbl_name) fk
    WHERE m.type = 'table';
"""

db.execute(query).fetchall()

[]

<database>
<tables>
[('cards_red', 'game_date', 'TEXT', 1, 0, None),
 ('cards_red', 'player_id', 'TEXT', 1, 0, None),
 ('cards_red', 'min_so', 'FLOAT', 0, 0, None),
 ('cards_yellow', 'game_date', 'TEXT', 1, 0, None),
 ('cards_yellow', 'player_id', 'TEXT', 1, 0, None),
 ('cards_yellow', 'min_yc', 'TEXT', 0, 0, None),
 ('cup_game_details', 'game_date', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_stage', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_round', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_section', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_leg_no', 'TEXT', 1, 0, None),
 ('cup_game_details', 'decider', 'TEXT', 1, 0, None),
 ('cup_game_details', 'cup_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_score', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_gf', 'TEXT', 1, 0, None),
 ('cup_game_details', 'agg_ga', 'TEXT', 1, 0, None),
 ('cup_game_details', 'gg_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'away_goal_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'pens_outcome', 'TEXT', 1, 0, None),
 ('cup_game_details', 'pens_score', 'TEXT', 1, 0, None),
 ('cup_game_details', 'pen_gf', 'TEXT', 1, 0, None),
 ('cup_game_details', 'pen_ga', 'TEXT', 1, 0, None),
 ('cup_game_details', 'game_length', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'outcome_desc', 'TEXT', 1, 0, None),
 ('cup_game_details', 'is_qualifying_round', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_group_stage', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_multi_leg', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_replay', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_extra_time', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_golden_goal', 'INTEGER', 1, 0, None),
 ('cup_game_details', 'is_pen_shootout', 'INTEGER', 1, 0, None),
 ('goals', 'game_date', 'TEXT', 1, 0, None),
 ('goals', 'player_id', 'TEXT', 1, 0, None),
 ('goals', 'goal_no', 'INTEGER', 1, 0, None),
 ('goals', 'goal_min', 'TEXT', 0, 0, None),
 ('goals', 'penalty', 'TEXT', 0, 0, None),
 ('goals', 'own_goal', 'FLOAT', 0, 0, None),
 ('league_tables', 'game_date', 'TEXT', 1, 0, None),
 ('league_tables', 'pos', 'INTEGER', 1, 0, None),
 ('league_tables', 'team_name', 'TEXT', 0, 0, None),
 ('league_tables', 'pld', 'INTEGER', 0, 0, None),
 ('league_tables', 'w', 'INTEGER', 0, 0, None),
 ('league_tables', 'd', 'INTEGER', 0, 0, None),
 ('league_tables', 'l', 'INTEGER', 0, 0, None),
 ('league_tables', 'gf', 'INTEGER', 0, 0, None),
 ('league_tables', 'ga', 'INTEGER', 0, 0, None),
 ('league_tables', 'pts', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'season', 'TEXT', 1, 0, None),
 ('league_tables_eos', 'pos', 'INTEGER', 1, 0, None),
 ('league_tables_eos', 'team_name', 'TEXT', 0, 0, None),
 ('league_tables_eos', 'pld', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'w', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'd', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'l', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'gf', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'ga', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'gd', 'INTEGER', 0, 0, None),
 ('league_tables_eos', 'pts', 'INTEGER', 0, 0, None),
 ('league_tiers', 'season', 'TEXT', 1, 0, None),
 ('league_tiers', 'league_tier', 'INTEGER', 0, 0, None),
 ('league_tiers', 'auto_prom_spots', 'INTEGER', 0, 0, None),
 ('league_tiers', 'playoff_spots', 'INTEGER', 0, 0, None),
 ('league_tiers', 'rel_spots', 'INTEGER', 0, 0, None),
 ('manager_reigns', 'manager_id', 'TEXT', 1, 0, None),
 ('manager_reigns', 'spell_no', 'INTEGER', 1, 0, None),
 ('manager_reigns', 'mgr_date_from', 'TEXT', 0, 0, None),
 ('manager_reigns', 'mgr_date_to', 'TEXT', 0, 0, None),
 ('manager_reigns', 'mgr_role', 'TEXT', 0, 0, None),
 ('managers', 'manager_id', 'TEXT', 1, 0, None),
 ('managers', 'surname', 'TEXT', 0, 0, None),
 ('managers', 'forename', 'TEXT', 0, 0, None),
 ('managers', 'manager_name', 'TEXT', 0, 0, None),
 ('managers', 'manager_dob', 'TEXT', 0, 0, None),
 ('managers', 'dob_year', 'TEXT', 0, 0, None),
 ('managers', 'birthplace', 'TEXT', 0, 0, None),
 ('managers', 'birth_country', 'TEXT', 0, 0, None),
 ('match_officials', 'game_date', 'TEXT', 1, 0, None),
 ('match_officials', 'role', 'TEXT', 1, 0, None),
 ('match_officials', 'surname', 'TEXT', 0, 0, None),
 ('match_officials', 'forename', 'TEXT', 0, 0, None),
 ('match_officials', 'name', 'TEXT', 0, 0, None),
 ('match_stats', 'game_date', 'TEXT', 1, 0, None),
 ('match_stats', 'team_name', 'TEXT', 1, 0, None),
 ('match_stats', 'team_venue', 'TEXT', 0, 0, None),
 ('match_stats', 'possessionpercentage', 'FLOAT', 0, 0, None),
 ('match_stats', 'shotstotal', 'INTEGER', 0, 0, None),
 ('match_stats', 'shotsontarget', 'INTEGER', 0, 0, None),
 ('match_stats', 'shotsofftarget', 'FLOAT', 0, 0, None),
 ('match_stats', 'shotsblocked', 'FLOAT', 0, 0, None),
 ('match_stats', 'shotssaved', 'INTEGER', 0, 0, None),
 ('match_stats', 'foulscommitted', 'INTEGER', 0, 0, None),
 ('match_stats', 'cornerswon', 'INTEGER', 0, 0, None),
 ('match_stats', 'touchesinbox', 'FLOAT', 0, 0, None),
 ('match_stats', 'aerialswon', 'FLOAT', 0, 0, None),
 ('match_stats', 'subsmade', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.shotstotal', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.shotsontarget', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.shotsofftarget', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.shotsblocked', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.cornerswon', 'TEXT', 0, 0, None),
 ('match_stats', 'attackoffside', 'TEXT', 0, 0, None),
 ('match_stats', 'attackcross', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.attemptsoutofbox', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.attemptsinbox', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.bigchancecreated', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.bigchancescored', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.hitwoodwork', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.shotsopenplayorfastbreak', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.touches', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.touchesinbox', 'TEXT', 0, 0, None),
 ('match_stats', 'distributionpass', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.accuratepass', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.backwardpass', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.forwardpass', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.passesright', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.passesleft', 'TEXT', 0, 0, None),
 ('match_stats', 'distributionlongballs', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.successfulfinalthirdpasses', 'TEXT', 0, 0, None),
 ('match_stats', 'distributionoffside', 'TEXT', 0, 0, None),
 ('match_stats', 'distribution.dispossessed', 'TEXT', 0, 0, None),
 ('match_stats', 'distributioncross', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.shotssaved', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.divingsave', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.keeperthrows', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.goalkicks', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.savedshotfromoutsidebox', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.foulscommitted', 'TEXT', 0, 0, None),
 ('match_stats', 'defencetackle', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.wontackle', 'TEXT', 0, 0, None),
 ('match_stats', 'defenceyellowcard', 'TEXT', 0, 0, None),
 ('match_stats', 'defenceclearance', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.headclearance', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.duelwon', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.duellost', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.aerialwon', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.aeriallost', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.savedshotfrominsidebox', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.bigchancemissed', 'TEXT', 0, 0, None),
 ('match_stats', 'goalkeeper.punches', 'TEXT', 0, 0, None),
 ('match_stats', 'attack.oneononeattempts', 'TEXT', 0, 0, None),
 ('match_stats', 'defence.clearanceoffline', 'TEXT', 0, 0, None),
 ('player_apps', 'game_date', 'TEXT', 1, 0, None),
 ('player_apps', 'player_id', 'TEXT', 1, 0, None),
 ('player_apps', 'role', 'TEXT', 0, 0, None),
 ('player_apps', 'shirt_no', 'INTEGER', 0, 0, None),
 ('player_positions', 'player_id', 'TEXT', 1, 0, None),
 ('player_positions', 'comp_rec_pos', 'TEXT', 0, 0, None),
 ('player_positions', 'soccerbase_pos', 'TEXT', 0, 0, None),
 ('player_positions', 'tm_pos_1', 'TEXT', 0, 0, None),
 ('player_positions', 'tm_pos_2', 'TEXT', 0, 0, None),
 ('player_positions', 'tm_pos_3', 'TEXT', 0, 0, None),
 ('players', 'player_id', 'TEXT', 1, 0, None),
 ('players', 'surname', 'TEXT', 0, 0, None),
 ('players', 'forename', 'TEXT', 0, 0, None),
 ('players', 'player_name', 'TEXT', 0, 0, None),
 ('players', 'player_dob', 'TEXT', 0, 0, None),
 ('results', 'game_date', 'TEXT', 1, 0, None),
 ('results', 'season', 'TEXT', 0, 0, None),
 ('results', 'game_no', 'INTEGER', 0, 0, None),
 ('results', 'opposition', 'TEXT', 0, 0, None),
 ('results', 'venue', 'TEXT', 0, 0, None),
 ('results', 'score', 'TEXT', 0, 0, None),
 ('results', 'outcome', 'TEXT', 0, 0, None),
 ('results', 'goals_for', 'FLOAT', 0, 0, None),
 ('results', 'goals_against', 'FLOAT', 0, 0, None),
 ('results', 'goal_diff', 'FLOAT', 0, 0, None),
 ('results', 'game_type', 'TEXT', 0, 0, None),
 ('results', 'comp_game_no', 'FLOAT', 0, 0, None),
 ('results', 'generic_comp', 'TEXT', 0, 0, None),
 ('results', 'competition', 'TEXT', 0, 0, None),
 ('results', 'league_pos', 'FLOAT', 0, 0, None),
 ('results', 'league_pts', 'FLOAT', 0, 0, None),
 ('results', 'attendance', 'FLOAT', 0, 0, None),
 ('results', 'weekday', 'TEXT', 0, 0, None),
 ('results', 'ko_time', 'TEXT', 0, 0, None),
 ('results', 'stadium', 'TEXT', 0, 0, None),
 ('results', 'referee', 'TEXT', 0, 0, None),
 ('sponsors_cup', 'season', 'TEXT', 1, 0, None),
 ('sponsors_cup', 'generic_comp_name', 'TEXT', 1, 0, None),
 ('sponsors_cup', 'sponsor_name', 'TEXT', 0, 0, None),
 ('sponsors_cup', 'sponsored_cup_name', 'TEXT', 0, 0, None),
 ('sponsors_cup', 'cup_spon_start', 'INTEGER', 0, 0, None),
 ('sponsors_cup', 'cup_spon_end', 'INTEGER', 0, 0, None),
 ('sponsors_league', 'season', 'TEXT', 1, 0, None),
 ('sponsors_league', 'generic_comp_name', 'TEXT', 1, 0, None),
 ('sponsors_league', 'sponsor_name', 'TEXT', 0, 0, None),
 ('sponsors_league', 'sponsored_lge_name', 'TEXT', 0, 0, None),
 ('sponsors_league', 'lge_spon_start', 'INTEGER', 0, 0, None),
 ('sponsors_league', 'lge_spon_end', 'INTEGER', 0, 0, None),
 ('squad_nos', 'season', 'TEXT', 1, 0, None),
 ('squad_nos', 'player_id', 'TEXT', 1, 0, None),
 ('squad_nos', 'shirt_no', 'INTEGER', 1, 0, None),
 ('subs_off', 'game_date', 'TEXT', 1, 0, None),
 ('subs_off', 'player_off_id', 'TEXT', 1, 0, None),
 ('subs_off', 'player_on_id', 'TEXT', 0, 0, None),
 ('subs_off', 'sub_min_off', 'FLOAT', 0, 0, None),
 ('subs_on', 'game_date', 'TEXT', 1, 0, None),
 ('subs_on', 'player_on_id', 'TEXT', 1, 0, None),
 ('subs_on', 'player_off_id', 'TEXT', 0, 0, None),
 ('subs_on', 'sub_min_on', 'FLOAT', 0, 0, None)]
</tables>

<relationships>
[Output from second query]
</relationships>

<sample_data>
[Optional: First few rows from key tables]
</sample_data>
</database>
